# **Sentiment Analysis using NLTK Vader**

# Installing and Importing Dependencies

In [6]:
#install and upgrade VADER
!pip install vaderSentiment
!pip install —-upgrade vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.8 MB/s eta 0:00:00
ERROR: Invalid requirement: '—-upgrade'


In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('all')
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#Magic command to print multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [5]:
#loading the data and first look
df = pd.read_csv("/content/yelp_sentiment.csv")
df
df.shape
df.info()
df.describe()
df.columns

Text  Sentiment
0    !....THE OWNERS REALLY REALLY need to quit bei...          0
1    (It wasn't busy either) Also, the building was...          0
2      (The bathroom is just next door and very nice.)          1
3    * Both the Hot & Sour & the Egg Flower Soups w...          1
4                                        2 Thumbs Up!!          1
..                                                 ...        ...
991       You cant go wrong with any of the food here.          1
992  You get incredibly fresh fish, prepared with c...          1
993                         You won't be disappointed.          1
994  Your servers suck, wait, correction, our serve...          0
995  Your staff spends more time talking to themsel...          0

[996 rows x 2 columns]

(996, 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       996 non-null    object
 1   Sentiment  996 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 15.7+ KB


Sentiment
count  996.000000
mean     0.500000
std      0.500251
min      0.000000
25%      0.000000
50%      0.500000
75%      1.000000
max      1.000000

Index(['Text', 'Sentiment'], dtype='object')

In [7]:
def clean_text(text):
    if isinstance(text, str):  # Check if 'text' is a string
            # Convert the text to lowercase
            text = text.lower()
            # Remove special Characters and digits
            text = re.sub(r'[^a-zA-Z\s]', '', text)
            # Remove URLs- r'http\S+|www.\S+' matches any website with http or www
            text = re.sub(r'http\S+|www.\S+', '', text)
            # Remove Punctuations- r'[“\w\s]' matches any character that is not a word character
            text = re.sub(r'[^\w\s]', '', text)
            # Remove Numbers- r'\d+' matches any digits (0-9)
            text = re.sub(r'\d+', '', text)
            # Remove extra white spaces- r'\s+' matches any whitespaces, tabs, lines
            text = re.sub(r'\s+', ' ', text).strip()
            # Remove @ mentions- It removes '@' followed by one or more alphanumeric characters
            text = re.sub(r'@\w+', '', text)
            # Remove hashtags- It removes '#' followed by one or more alphanumeric characters
            text = re.sub(r'#\w+', '', text)
            # Remove special characters- Removes anything which is not alphanumeric characters
            text = re.sub(r'[^\w\s]', '', text)
            # Tokenize the text
            tokens = nltk.word_tokenize(text)
            # Remove stop words
            stop_words = set(stopwords.words('english'))
            #tokens = [X for X in tokens if X not in stop_words]
            # Lemmatize the text
            lemmatizer = WordNetLemmatizer()
            tokens = [lemmatizer.lemmatize(x) for x in tokens]
            # Join the tokens back into a string
            text = ' '.join(tokens)
    return text

In [8]:
# apply the function to the original 'Text' and create a new column with clean text
df['Text_Clean'] = df['Text'].apply(clean_text)
df.head()

Text  Sentiment  \
0  !....THE OWNERS REALLY REALLY need to quit bei...          0   
1  (It wasn't busy either) Also, the building was...          0   
2    (The bathroom is just next door and very nice.)          1   
3  * Both the Hot & Sour & the Egg Flower Soups w...          1   
4                                      2 Thumbs Up!!          1   

                                          Text_Clean  
0  the owner really really need to quit being soo...  
1  it wasnt busy either also the building wa free...  
2       the bathroom is just next door and very nice  
3  both the hot sour the egg flower soup were abs...  
4                                           thumb up

In [15]:
# Model Building
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Instantiate the VADER sentiment Analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# Create a function to classify sentiment into positive and Neutral
def get_sentiment_label(score, threshold=0.0):
    if score > threshold:
        return 1
    else:
        return 0

# Split data into train and test sets
x = df['Text']
y = df['Sentiment']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=23)

# Optimize score threshold
best_accuracy = 0.0
best_threshold = 0.0
for threshold in [0.1 * i for i in range(-10, 11)]:
    y_pred = x_test.apply(lambda x: get_sentiment_label(sentiment_analyzer.polarity_scores(x)['compound'], threshold=threshold))
    accuracy = accuracy_score(y_test, y_pred)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = threshold

# Print the best threshold value
print("Best Threshold:", best_threshold)

# Apply the best threshold to sentiment analysis and create a new variable in the original data frame
df['sentiment_label_VADER'] = df['Text'].apply(lambda x: get_sentiment_label(sentiment_analyzer.polarity_scores(x)['compound'], threshold=best_threshold))

# Calculate accuracy
accuracy = accuracy_score(df['Sentiment'], df['sentiment_label_VADER'])
print('Accuracy: {:.2f}%'.format(accuracy * 100))


Best Threshold: 0.4
Accuracy: 82.13%


# **Naive Bayes Classifier**

In [17]:
#Export csv file to visualize in spreadsheet
df.to_csv('cleaned_data_yelp.csv')

In [19]:
# Now let's begin with N8 modeling by creatings features (x) and label (y)
x = df.Text_Clean
y = df.Sentiment
x.head()
y.head()
x.shape
print()
y.shape

0    the owner really really need to quit being soo...
1    it wasnt busy either also the building wa free...
2         the bathroom is just next door and very nice
3    both the hot sour the egg flower soup were abs...
4                                             thumb up
Name: Text_Clean, dtype: object

0    0
1    0
2    1
3    1
4    1
Name: Sentiment, dtype: int64

(996,)

(996,)

In [28]:
# Vectorize the x by TFIDF vectorizer
vectorizer = TfidfVectorizer()
x_vect = vectorizer.fit_transform(x)
print(x_vect.shape)
print(type(x_vect))
print(vectorizer.get_feature_names_out()[0:10]) # print the first 16 words/features
print (x_vect) # print each feature and the TFIDF score
print(vectorizer.get_feature_names_out()[995]) # print TFIDF score for word sitting at 995th position

(996, 1911)
<class 'scipy.sparse._csr.csr_matrix'>
['about' 'above' 'absolute' 'absolutely' 'absolutley' 'accident'
 'accommodation' 'accomodate' 'accountant' 'ache']
  (0, 1136)	0.17913847993299328
  (0, 1109)	0.12315475148848944
  (0, 1186)	0.2533582524912772
  (0, 1741)	0.22183915508744206
  (0, 838)	0.1278607878918281
  (0, 1423)	0.21774402052576303
  (0, 644)	0.2802993789438084
  (0, 1076)	0.14101577572984839
  (0, 1887)	0.2645398302418908
  (0, 1663)	0.20198447182384552
  (0, 930)	0.2533582524912772
  (0, 269)	0.2645398302418908
  (0, 1534)	0.2802993789438084
  (0, 143)	0.20198447182384552
  (0, 1314)	0.2802993789438084
  (0, 1694)	0.10475908567386075
  (0, 1087)	0.22641712603874595
  (0, 1336)	0.3403570513353209
  (0, 1173)	0.22641712603874595
  (0, 1660)	0.0699604427288624
  (1, 306)	0.3384551738028034
  (1, 646)	0.4356894613600644
  (1, 1802)	0.14138515706203436
  (1, 211)	0.4111932626489262
  (1, 30)	0.27625065524929154
  :	:
  (992, 676)	0.3019332962474302
  (992, 1866)	0.22

In [29]:
# Split data into training and testing
x_train, x_test, y_train, y_test = train_test_split(x_vect, y)
x_train.shape
x_test.shape
y_train.shape
y_test.shape

(747, 1911)

(249, 1911)

(747,)

(249,)

In [30]:
# Build the NB model and make predictions on the test data to get classification report
from sklearn.metrics import classification_report as cr
clf = MultinomialNB() #multi class naive bayes - naive bayes works best on text classification
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(cr(y_test, y_pred))

MultinomialNB()

              precision    recall  f1-score   support

           0       0.81      0.81      0.81       124
           1       0.82      0.82      0.82       125

    accuracy                           0.82       249
   macro avg       0.82      0.82      0.82       249
weighted avg       0.82      0.82      0.82       249



In [31]:
# Make predictions for the entire data and append into the original data
df['Predicted'] = clf.predict(x_vect)
df.sample(10)

Text  Sentiment  \
182                    Great food and awesome service!          1   
229   I can assure you that you won't be disappointed.          1   
853                  This place is great!!!!!!!!!!!!!!          1   
722            the presentation of the food was awful.          0   
597  Sorry, I will not be getting food from here an...          0   
811  They could serve it with just the vinaigrette ...          1   
92                       But then they came back cold.          0   
9                     A fantastic neighborhood gem !!!          1   
852                                this place is good.          1   
539                             Penne vodka excellent!          1   

                                            Text_Clean  sentiment_label_VADER  \
182                     great food and awesome service                      1   
229     i can assure you that you wont be disappointed                      1   
853                                this place is great                      0   
722              the presentation of the food wa awful                      0   
597  sorry i will not be getting food from here any...                      0   
811  they could serve it with just the vinaigrette ...                      1   
92                        but then they came back cold                      0   
9                         a fantastic neighborhood gem                      1   
852                                 this place is good                      1   
539                              penne vodka excellent                      1   

     Predicted  
182          1  
229          0  
853          1  
722          0  
597          0  
811          1  
92           0  
9            1  
852          1  
539          1